In [21]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

In [22]:
user_info = pd.read_csv('./user_info.csv')
user_info['reg_date'] = pd.to_datetime(user_info['reg_date']).dt.date
user_info = user_info[['user_id','channel_id','source_id','reg_date']]
user_info.drop_duplicates(inplace=True)

In [23]:
source_1 = pd.read_csv('./source_1.csv')
source_2 = pd.read_csv('./source_2.csv')
source_1.rename(columns={'chl_user_id':'channel_id'},inplace=True)
source_1.drop_duplicates(subset=['channel_id','source_id'],inplace=True)
source = pd.merge(source_1,source_2,on=['channel_id','source_id'],how='left',validate='one_to_many')
source.dropna(inplace=True)
source['create_time'] = pd.to_datetime(source['create_time']).dt.date
source['tdate'] = pd.to_datetime(source['tdate']).dt.date
source.rename(columns={'tdate':'reg_date'},inplace=True)
source.drop_duplicates(subset=['channel_id','source_id','reg_date'],inplace=True)

In [24]:
source_info = pd.merge(source,user_info,on=['channel_id','source_id','reg_date'],how='left',validate='one_to_many')
source_info.dropna(inplace=True)

In [26]:
role_info_1d = pd.read_pickle('./role_info_1d.pkl')
df_pay = role_info_1d[role_info_1d['pay_sum']!=0]
df_nopay = role_info_1d[role_info_1d['pay_sum']==0]
df_nopay_online_n = df_nopay[(df_nopay['role_created_online']<=600) & (df_nopay['role_created_login_num']<=2)&(df_nopay['max_role_level']<=10)]
df_nopay_online_y = df_nopay.loc[np.setdiff1d(df_nopay.index, df_nopay_online_n.index),:]
# 标记角户质量不付费不在线0;不付费在线1;付费2
df_nopay_online_n['role_type'] = 0
df_nopay_online_y['role_type'] = 1
df_pay['role_type'] = 2
print('0类的角色占比：%.4f' % (df_nopay_online_n.shape[0]/role_info_1d.shape[0]))
print('1类的角色占比：%.4f' % (df_nopay_online_y.shape[0]/role_info_1d.shape[0]))
print('2类的角色占比：%.4f' % (df_pay.shape[0]/role_info_1d.shape[0]))

0类的角色占比：0.6122
1类的角色占比：0.3723
2类的角色占比：0.0155


In [27]:
df_nopay_online_n['predict_30_pay'] = 0
df_nopay_online_y['predict_30_pay'] = 0
lgb_r_1d = joblib.load('./lgb_r_1d.pkl')
select_features = ['role_created_login_num', 'role_created_active', 'role_created_online', 'max_role_level', 'ip_num', 'pay_num', 'pay_sum',
                           'active_0-8', 'active_8-12', 'active_12-14', 'active_14-18', 'active_18-24', 'pay_grade_1', 'pay_grade_2',
                           'pay_grade_3', 'pay_grade_4', 'pay_grade_5', 'pay_grade_6',
                           'pay_rate', 'pay_avg']
df_pay['predict_30_pay'] = np.expm1(lgb_r_1d.predict(df_pay[select_features]))*2.1

In [28]:
df_pay['predict_30_pay'].sum()/role_info_1d['role_created_30_pay_sum'].sum()

0.8452350400226288

In [30]:
df_predict = pd.concat([df_pay,df_nopay_online_n,df_nopay_online_y],axis=0)
source_info['user_id'] = source_info['user_id'].astype(np.int64)
data = pd.merge(source_info,df_predict,on='user_id',how='left',validate='one_to_many')
data.dropna(inplace=True)

In [32]:
groups = data.groupby(['channel_id','source_id','reg_date'])
# 统计计划（天）的30天真实回款
temp = groups['role_created_30_pay_sum'].agg([('source_30_pay_sum','sum')]).reset_index()
df = pd.merge(data,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')
# 统计计划（天）的30天预测回款
temp = groups['predict_30_pay'].agg([('source_30_predict_pay','sum')]).reset_index()
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')

In [33]:
def count_0(list):
    count = 0
    for num in list:
        if num ==0:
            count += 1
    return count

def count_1(list):
    count = 0
    for num in list:
        if num ==1:
            count += 1
    return count

def count_2(list):
    count = 0
    for num in list:
        if num ==2:
            count += 1
    return count

In [34]:
temp = groups['role_type'].apply(count_0).reset_index().rename(columns = {'role_type':'type_0_num'})
temp['reg_date'] = temp['reg_date'].astype(str)
df['reg_date'] = df['reg_date'].astype(str)
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')
temp = groups['role_type'].apply(count_1).reset_index().rename(columns = {'role_type':'type_1_num'})
temp['reg_date'] = temp['reg_date'].astype(str)
df['reg_date'] = df['reg_date'].astype(str)
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')
temp = groups['role_type'].apply(count_2).reset_index().rename(columns = {'role_type':'type_2_num'})
temp['reg_date'] = temp['reg_date'].astype(str)
df['reg_date'] = df['reg_date'].astype(str)
df = pd.merge(df,temp,on=['channel_id','source_id','reg_date'],how='left',validate='many_to_one')
df['type_0_rate'] = df['type_0_num']/(df['type_0_num']+df['type_1_num']+df['type_2_num'])
df['type_1_rate'] = df['type_1_num']/(df['type_0_num']+df['type_1_num']+df['type_2_num'])
df['type_2_rate'] = df['type_2_num']/(df['type_0_num']+df['type_1_num']+df['type_2_num'])
df['source_role_num'] = df['type_0_num']+df['type_1_num']+df['type_2_num']

In [35]:
df_source = df.drop_duplicates(subset=['channel_id','source_id','reg_date'])
col = ['channel_id','source_id','create_time','reg_date','amount','paper_use_amount','click_num','down_num','pv',
       'create_role_num','source_30_pay_sum','source_30_predict_pay','type_0_num','type_1_num','type_2_num','type_0_rate',
       'type_1_rate','type_2_rate','source_role_num']
df_source = df_source[col]

In [36]:
df_source['30_ROI'] = df_source['source_30_pay_sum']/df_source['amount']
df_source['30_ROI_predict'] = df_source['source_30_predict_pay']/df_source['amount']
# df_source_no_amount = df_source[df_source['amount']==0]
df_source_amount = df_source[df_source['amount']>500]

In [38]:
# 消耗大于500但30天回款为0的计划,消耗占比（消耗大于500的计划数）
print(df_source_amount[df_source_amount['30_ROI'] == 0]['amount'].sum()/df_source_amount['amount'].sum())
# 消耗大于500但30天回款为0的计划数占比（消耗大于500的计划数）
print(df_source_amount[df_source_amount['30_ROI'] == 0].shape[0]/df_source_amount.shape[0])

0.1526535339253292
0.3999621785173979


In [39]:
df_1 = df_source_amount[['type_0_rate','type_1_rate','type_2_rate','30_ROI_predict','30_ROI']]
df_1['30_ROI'] = df_1['30_ROI'].apply(lambda x:0 if x==0 else 1)
df_1.head()

,type_0_rate,type_1_rate,type_2_rate,30_ROI_predict,30_ROI
0,0.549020,0.450980,0.000000,0.000000,0
59,0.666667,0.333333,0.000000,0.000000,0
173,0.674419,0.325581,0.000000,0.000000,0
223,0.766839,0.227979,0.005181,0.033252,1
440,0.774869,0.222513,0.002618,0.223788,1


In [40]:
df_1.corr()

,type_0_rate,type_1_rate,type_2_rate,30_ROI_predict,30_ROI
type_0_rate,1.000000,-0.981463,-0.184280,-0.013606,0.042737
type_1_rate,-0.981463,1.000000,-0.007504,-0.024028,-0.096319
type_2_rate,-0.184280,-0.007504,1.000000,0.194220,0.270985
30_ROI_predict,-0.013606,-0.024028,0.194220,1.000000,0.190295
30_ROI,0.042737,-0.096319,0.270985,0.190295,1.000000


In [41]:
df_1['30_ROI'].value_counts()

1    15865
0    10575
Name: 30_ROI, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,f1_score,accuracy_score,precision_score,classification_report
import lightgbm as lgb

In [43]:
df_train,df_test = train_test_split(df_1,test_size=0.2)
X_val, x_test, Y_val, y_test = train_test_split(df_train.drop('30_ROI',axis=1),df_train['30_ROI'],test_size=0.3)

In [48]:
# lgb的调参与交叉验证
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

# 设置参数初始值，不含交叉验证参数
print('设置参数')
params = {
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric':'auc',
    'nthread':4,
    'learning_rate':0.01
}

# 交叉验证（调参）
print('交叉验证')
max_auc = float('0')
best_params = {}

# 准确率
print('调参1：提高准确率')
for num_leaves in range(5, 100, 5):
    for max_depth in range(3, 8, 1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True     
                            )
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
        
        if mean_auc >= max_auc:
            max_auc = mean_auc
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth

if 'num_leaves' and 'max_depth' in best_params.keys():
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']

# 过拟合
print('调参2：降低过拟合')
for max_bin in range(5, 256, 10):
    for min_data_in_leaf in range(1, 102, 10):
        params['max_bin'] = max_bin
        params['min_data_in_leaf'] = min_data_in_leaf
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
        
        if mean_auc >= max_auc:
            max_auc = mean_auc
            best_params['max_bin'] = max_bin
            best_params['min_data_in_leaf'] = min_data_in_leaf

if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['max_bin'] = best_params['max_bin']
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']

print('调参3：降低过拟合')
for feature_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    for bagging_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq

            cv_results = lgb.cv(
                                params,
                                train_data,
                                seed=1,
                                nfold=5,
                                metrics=['auc'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
            mean_auc = pd.Series(cv_results['auc-mean']).max()
            boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()

            if mean_auc >= max_auc:
                max_auc = mean_auc
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']

print('调参4：降低过拟合')

for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            train_data,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
                
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
        if mean_auc >= max_auc:
            max_auc=mean_auc
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']

print("调参5：降低过拟合2")
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                        params,
                        train_data,
                        seed=1,
                        nfold=5,
                        metrics=['auc'],
                        early_stopping_rounds=10,
                        verbose_eval=True
                        )
            
    mean_auc = pd.Series(cv_results['auc-mean']).max()
    boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
    if mean_auc >= max_auc:
        max_auc=mean_auc
        
        best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
print(best_params)

设置参数
交叉验证
调参1：提高准确率
[1]	cv_agg's auc: 0.905421 + 0.00567019
[2]	cv_agg's auc: 0.905505 + 0.0058013
[3]	cv_agg's auc: 0.907693 + 0.0055798
[4]	cv_agg's auc: 0.907798 + 0.00571165
[5]	cv_agg's auc: 0.908308 + 0.00521331
[6]	cv_agg's auc: 0.909379 + 0.00474605
[7]	cv_agg's auc: 0.909379 + 0.00474605
[8]	cv_agg's auc: 0.909379 + 0.00474605
[9]	cv_agg's auc: 0.911331 + 0.00638937
[10]	cv_agg's auc: 0.911331 + 0.00638937
[11]	cv_agg's auc: 0.911616 + 0.00640725
[12]	cv_agg's auc: 0.911616 + 0.00640725
[13]	cv_agg's auc: 0.911616 + 0.00640725
[14]	cv_agg's auc: 0.911616 + 0.00640725
[15]	cv_agg's auc: 0.911691 + 0.00645025
[16]	cv_agg's auc: 0.911691 + 0.00645025
[17]	cv_agg's auc: 0.91178 + 0.00653346
[18]	cv_agg's auc: 0.912084 + 0.00678112
[19]	cv_agg's auc: 0.912089 + 0.00678768
[20]	cv_agg's auc: 0.912476 + 0.00615709
[21]	cv_agg's auc: 0.912489 + 0.00616663
[22]	cv_agg's auc: 0.912489 + 0.00616663
[23]	cv_agg's auc: 0.912489 + 0.00616663
[24]	cv_agg's auc: 0.912519 + 0.00620737
[25]	cv_

[1]	cv_agg's auc: 0.911204 + 0.00551935
[2]	cv_agg's auc: 0.91122 + 0.00551219
[3]	cv_agg's auc: 0.911397 + 0.00550393
[4]	cv_agg's auc: 0.911279 + 0.00545599
[5]	cv_agg's auc: 0.911746 + 0.00552997
[6]	cv_agg's auc: 0.911746 + 0.00552997
[7]	cv_agg's auc: 0.91175 + 0.00553535
[8]	cv_agg's auc: 0.911878 + 0.00547067
[9]	cv_agg's auc: 0.911878 + 0.00547124
[10]	cv_agg's auc: 0.911878 + 0.00547124
[11]	cv_agg's auc: 0.911878 + 0.00547124
[12]	cv_agg's auc: 0.911878 + 0.00547124
[13]	cv_agg's auc: 0.912046 + 0.0055878
[14]	cv_agg's auc: 0.912046 + 0.0055878
[15]	cv_agg's auc: 0.91253 + 0.00619376
[16]	cv_agg's auc: 0.912587 + 0.00622875
[17]	cv_agg's auc: 0.912587 + 0.00622875
[18]	cv_agg's auc: 0.912965 + 0.00580658
[19]	cv_agg's auc: 0.912947 + 0.00579427
[20]	cv_agg's auc: 0.913028 + 0.0058212
[21]	cv_agg's auc: 0.913028 + 0.0058212
[22]	cv_agg's auc: 0.913028 + 0.0058212
[23]	cv_agg's auc: 0.913022 + 0.00581265
[24]	cv_agg's auc: 0.913153 + 0.00577548
[25]	cv_agg's auc: 0.913158 + 0.0

[1]	cv_agg's auc: 0.914008 + 0.0047453
[2]	cv_agg's auc: 0.914008 + 0.0047453
[3]	cv_agg's auc: 0.914232 + 0.00501654
[4]	cv_agg's auc: 0.91418 + 0.004951
[5]	cv_agg's auc: 0.914942 + 0.00529796
[6]	cv_agg's auc: 0.914944 + 0.00528191
[7]	cv_agg's auc: 0.915025 + 0.00534962
[8]	cv_agg's auc: 0.915243 + 0.00531871
[9]	cv_agg's auc: 0.915519 + 0.00551683
[10]	cv_agg's auc: 0.915527 + 0.00552561
[11]	cv_agg's auc: 0.915616 + 0.0055831
[12]	cv_agg's auc: 0.915619 + 0.00555902
[13]	cv_agg's auc: 0.915704 + 0.00564323
[14]	cv_agg's auc: 0.915745 + 0.00568499
[15]	cv_agg's auc: 0.915726 + 0.0056867
[16]	cv_agg's auc: 0.915733 + 0.0057251
[17]	cv_agg's auc: 0.915712 + 0.00571377
[18]	cv_agg's auc: 0.91565 + 0.00591657
[19]	cv_agg's auc: 0.915815 + 0.0057162
[20]	cv_agg's auc: 0.915666 + 0.00589606
[21]	cv_agg's auc: 0.916014 + 0.00548617
[22]	cv_agg's auc: 0.916082 + 0.00552764
[23]	cv_agg's auc: 0.916089 + 0.0055374
[24]	cv_agg's auc: 0.916085 + 0.00553509
[25]	cv_agg's auc: 0.916282 + 0.0053

[62]	cv_agg's auc: 0.913974 + 0.00533705
[63]	cv_agg's auc: 0.913974 + 0.00533705
[64]	cv_agg's auc: 0.913974 + 0.00533705
[65]	cv_agg's auc: 0.914139 + 0.00538441
[66]	cv_agg's auc: 0.914139 + 0.00538441
[67]	cv_agg's auc: 0.914139 + 0.00538441
[68]	cv_agg's auc: 0.914139 + 0.00538441
[69]	cv_agg's auc: 0.914139 + 0.00538441
[70]	cv_agg's auc: 0.914139 + 0.00538441
[71]	cv_agg's auc: 0.914139 + 0.00538441
[72]	cv_agg's auc: 0.914139 + 0.00538441
[73]	cv_agg's auc: 0.914139 + 0.00538441
[74]	cv_agg's auc: 0.914139 + 0.00538441
[75]	cv_agg's auc: 0.914139 + 0.00538441
[1]	cv_agg's auc: 0.912058 + 0.00498928
[2]	cv_agg's auc: 0.912058 + 0.00498928
[3]	cv_agg's auc: 0.91214 + 0.00496819
[4]	cv_agg's auc: 0.912216 + 0.00504459
[5]	cv_agg's auc: 0.912615 + 0.00500178
[6]	cv_agg's auc: 0.912615 + 0.00500178
[7]	cv_agg's auc: 0.912742 + 0.00490761
[8]	cv_agg's auc: 0.912742 + 0.00490761
[9]	cv_agg's auc: 0.912763 + 0.00492722
[10]	cv_agg's auc: 0.912763 + 0.00492722
[11]	cv_agg's auc: 0.91276

[10]	cv_agg's auc: 0.916934 + 0.00575264
[11]	cv_agg's auc: 0.916923 + 0.00572118
[12]	cv_agg's auc: 0.917078 + 0.00593187
[13]	cv_agg's auc: 0.917141 + 0.00603836
[14]	cv_agg's auc: 0.917102 + 0.00598273
[15]	cv_agg's auc: 0.917074 + 0.00611487
[16]	cv_agg's auc: 0.917531 + 0.00606268
[17]	cv_agg's auc: 0.917519 + 0.00612884
[18]	cv_agg's auc: 0.91752 + 0.0061582
[19]	cv_agg's auc: 0.917518 + 0.00613221
[20]	cv_agg's auc: 0.917722 + 0.00607049
[21]	cv_agg's auc: 0.917651 + 0.00617769
[22]	cv_agg's auc: 0.917596 + 0.00616652
[23]	cv_agg's auc: 0.917854 + 0.00598526
[24]	cv_agg's auc: 0.918231 + 0.00579668
[25]	cv_agg's auc: 0.918152 + 0.00593415
[26]	cv_agg's auc: 0.918214 + 0.0060323
[27]	cv_agg's auc: 0.918306 + 0.00608441
[28]	cv_agg's auc: 0.918331 + 0.00608897
[29]	cv_agg's auc: 0.918422 + 0.00603956
[30]	cv_agg's auc: 0.91839 + 0.00603563
[31]	cv_agg's auc: 0.918394 + 0.00601702
[32]	cv_agg's auc: 0.918386 + 0.00594623
[33]	cv_agg's auc: 0.918508 + 0.00591692
[34]	cv_agg's auc: 0

...

[26]	cv_agg's auc: 0.918457 + 0.00504892
[27]	cv_agg's auc: 0.91877 + 0.00515074
[28]	cv_agg's auc: 0.918771 + 0.00515048
[29]	cv_agg's auc: 0.918811 + 0.00515028
[30]	cv_agg's auc: 0.918812 + 0.00515005
[31]	cv_agg's auc: 0.919195 + 0.00495371
[32]	cv_agg's auc: 0.919196 + 0.00495366
[33]	cv_agg's auc: 0.919228 + 0.00477821
[34]	cv_agg's auc: 0.919228 + 0.00477845
[35]	cv_agg's auc: 0.919543 + 0.00496128
[36]	cv_agg's auc: 0.919543 + 0.00496157
[37]	cv_agg's auc: 0.919587 + 0.00494646
[38]	cv_agg's auc: 0.919587 + 0.00494639
[39]	cv_agg's auc: 0.919771 + 0.00512073
[40]	cv_agg's auc: 0.919772 + 0.00512018
[41]	cv_agg's auc: 0.919627 + 0.00514441
[42]	cv_agg's auc: 0.919628 + 0.00514385
[43]	cv_agg's auc: 0.919584 + 0.00498773
[44]	cv_agg's auc: 0.919585 + 0.00498726
[45]	cv_agg's auc: 0.919553 + 0.00499099
[46]	cv_agg's auc: 0.919554 + 0.00499044
[47]	cv_agg's auc: 0.919841 + 0.00509499
[48]	cv_agg's auc: 0.919841 + 0.00509499
[49]	cv_agg's auc: 0.919914 + 0.00514393
[50]	cv_agg's auc

[38]	cv_agg's auc: 0.919523 + 0.00514193
[39]	cv_agg's auc: 0.919595 + 0.00524249
[40]	cv_agg's auc: 0.919595 + 0.0052424
[41]	cv_agg's auc: 0.919683 + 0.00516063
[42]	cv_agg's auc: 0.919683 + 0.00516054
[43]	cv_agg's auc: 0.919611 + 0.00508391
[44]	cv_agg's auc: 0.919611 + 0.00508344
[45]	cv_agg's auc: 0.919704 + 0.00476196
[46]	cv_agg's auc: 0.919705 + 0.00476151
[47]	cv_agg's auc: 0.920029 + 0.00482821
[48]	cv_agg's auc: 0.920029 + 0.00482775
[49]	cv_agg's auc: 0.920179 + 0.00486946
[50]	cv_agg's auc: 0.920179 + 0.00486984
[51]	cv_agg's auc: 0.920388 + 0.00488835
[52]	cv_agg's auc: 0.920388 + 0.00488872
[53]	cv_agg's auc: 0.920523 + 0.00488706
[54]	cv_agg's auc: 0.920523 + 0.00488706
[55]	cv_agg's auc: 0.920663 + 0.00500292
[56]	cv_agg's auc: 0.920663 + 0.00500292
[57]	cv_agg's auc: 0.920576 + 0.00487854
[58]	cv_agg's auc: 0.920576 + 0.00487854
[59]	cv_agg's auc: 0.920568 + 0.00495497
[60]	cv_agg's auc: 0.920568 + 0.00495497
[61]	cv_agg's auc: 0.920609 + 0.00503566
[62]	cv_agg's auc

[100]	cv_agg's auc: 0.921222 + 0.00504726
[1]	cv_agg's auc: 0.702404 + 0.00754231
[2]	cv_agg's auc: 0.915872 + 0.00537049
[3]	cv_agg's auc: 0.911529 + 0.00424697
[4]	cv_agg's auc: 0.915895 + 0.00547588
[5]	cv_agg's auc: 0.914905 + 0.00595381
[6]	cv_agg's auc: 0.915677 + 0.00583865
[7]	cv_agg's auc: 0.91678 + 0.00602953
[8]	cv_agg's auc: 0.917026 + 0.00605214
[9]	cv_agg's auc: 0.917222 + 0.00530772
[10]	cv_agg's auc: 0.917361 + 0.00540315
[11]	cv_agg's auc: 0.918116 + 0.00507517
[12]	cv_agg's auc: 0.918134 + 0.00508102
[13]	cv_agg's auc: 0.91798 + 0.00499701
[14]	cv_agg's auc: 0.917994 + 0.00499948
[15]	cv_agg's auc: 0.918058 + 0.00480148
[16]	cv_agg's auc: 0.918068 + 0.00479974
[17]	cv_agg's auc: 0.918163 + 0.00483066
[18]	cv_agg's auc: 0.918168 + 0.00483425
[19]	cv_agg's auc: 0.918317 + 0.00481124
[20]	cv_agg's auc: 0.918318 + 0.00481215
[21]	cv_agg's auc: 0.91848 + 0.00489351
[22]	cv_agg's auc: 0.918481 + 0.00489389
[23]	cv_agg's auc: 0.918494 + 0.0050948
[24]	cv_agg's auc: 0.918495 

[10]	cv_agg's auc: 0.917313 + 0.00545588
[11]	cv_agg's auc: 0.918054 + 0.0051237
[12]	cv_agg's auc: 0.918073 + 0.00512949
[13]	cv_agg's auc: 0.917967 + 0.00500008
[14]	cv_agg's auc: 0.917981 + 0.00500251
[15]	cv_agg's auc: 0.918057 + 0.00477586
[16]	cv_agg's auc: 0.918068 + 0.00477408
[17]	cv_agg's auc: 0.918013 + 0.00490299
[18]	cv_agg's auc: 0.918018 + 0.00490559
[19]	cv_agg's auc: 0.918294 + 0.00478939
[20]	cv_agg's auc: 0.918296 + 0.0047903
[21]	cv_agg's auc: 0.918449 + 0.00484822
[22]	cv_agg's auc: 0.91845 + 0.00484859
[23]	cv_agg's auc: 0.91847 + 0.00509309
[24]	cv_agg's auc: 0.918471 + 0.00509358
[25]	cv_agg's auc: 0.918523 + 0.0049343
[26]	cv_agg's auc: 0.918524 + 0.00493408
[27]	cv_agg's auc: 0.918761 + 0.00520609
[28]	cv_agg's auc: 0.918761 + 0.00520583
[29]	cv_agg's auc: 0.918718 + 0.00515427
[30]	cv_agg's auc: 0.918718 + 0.00515424
[31]	cv_agg's auc: 0.919156 + 0.00497221
[32]	cv_agg's auc: 0.919156 + 0.00497236
[33]	cv_agg's auc: 0.919497 + 0.00506867
[34]	cv_agg's auc: 0.

[1]	cv_agg's auc: 0.702404 + 0.00754231
[2]	cv_agg's auc: 0.915872 + 0.00537049
[3]	cv_agg's auc: 0.911529 + 0.00424697
[4]	cv_agg's auc: 0.915895 + 0.00547588
[5]	cv_agg's auc: 0.914902 + 0.00595751
[6]	cv_agg's auc: 0.915674 + 0.00584248
[7]	cv_agg's auc: 0.916764 + 0.00604775
[8]	cv_agg's auc: 0.91701 + 0.00607098
[9]	cv_agg's auc: 0.917175 + 0.00536066
[10]	cv_agg's auc: 0.917313 + 0.00545588
[11]	cv_agg's auc: 0.918065 + 0.00510846
[12]	cv_agg's auc: 0.918083 + 0.00511424
[13]	cv_agg's auc: 0.917967 + 0.00499923
[14]	cv_agg's auc: 0.917982 + 0.00500166
[15]	cv_agg's auc: 0.918047 + 0.00479148
[16]	cv_agg's auc: 0.918058 + 0.0047897
[17]	cv_agg's auc: 0.918002 + 0.00491938
[18]	cv_agg's auc: 0.918008 + 0.00492197
[19]	cv_agg's auc: 0.918273 + 0.00482344
[20]	cv_agg's auc: 0.918274 + 0.00482436
[21]	cv_agg's auc: 0.91844 + 0.00486217
[22]	cv_agg's auc: 0.918442 + 0.00486254
[23]	cv_agg's auc: 0.91848 + 0.00507669
[24]	cv_agg's auc: 0.918481 + 0.00507718
[25]	cv_agg's auc: 0.91851 + 

In [50]:
# 不调参
params = {
"objective": "binary",
"boosting_type": "gbdt",
"learning_rate": 0.03,
"max_depth":7,
"num_leaves":20,
"min_data_in_leaf":101,
"min_split_gain":0.0,
# "min_child_samples":15,
"feature_fraction": 0.6,
"bagging_fraction":0.6,
"bagging_freq":5,
"lambda_l1":1.0,
"lambda_l2":0.7,
"metric": "auc",
}

train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_b = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's auc: 0.701566	valid_1's auc: 0.690227
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.917232	valid_1's auc: 0.91301
[3]	training's auc: 0.915303	valid_1's auc: 0.910942
[4]	training's auc: 0.917389	valid_1's auc: 0.913212
[5]	training's auc: 0.918123	valid_1's auc: 0.912527
[6]	training's auc: 0.918489	valid_1's auc: 0.913319
[7]	training's auc: 0.918754	valid_1's auc: 0.912971
[8]	training's auc: 0.918911	valid_1's auc: 0.913342
[9]	training's auc: 0.918632	valid_1's auc: 0.912874
[10]	training's auc: 0.918749	valid_1's auc: 0.913157
[11]	training's auc: 0.918928	valid_1's auc: 0.913468
[12]	training's auc: 0.918931	valid_1's auc: 0.913473
[13]	training's auc: 0.919466	valid_1's auc: 0.914375
[14]	training's auc: 0.919467	valid_1's auc: 0.914376
[15]	training's auc: 0.920062	valid_1's auc: 0.914752
[16]	training's auc: 0.920063	valid_1's auc: 0.914753
[17]	training's auc: 0.920433	valid_1's auc: 0.914956
[18]	training's auc: 0.920434	

[151]	training's auc: 0.929377	valid_1's auc: 0.922011
[152]	training's auc: 0.929377	valid_1's auc: 0.922011
[153]	training's auc: 0.929452	valid_1's auc: 0.921974
[154]	training's auc: 0.929452	valid_1's auc: 0.921974
[155]	training's auc: 0.929496	valid_1's auc: 0.922024
[156]	training's auc: 0.929496	valid_1's auc: 0.922024
[157]	training's auc: 0.929658	valid_1's auc: 0.921998
[158]	training's auc: 0.929658	valid_1's auc: 0.921998
[159]	training's auc: 0.929729	valid_1's auc: 0.922048
[160]	training's auc: 0.929729	valid_1's auc: 0.922048
[161]	training's auc: 0.929808	valid_1's auc: 0.92203
[162]	training's auc: 0.929808	valid_1's auc: 0.92203
[163]	training's auc: 0.929883	valid_1's auc: 0.922007
[164]	training's auc: 0.929883	valid_1's auc: 0.922007
[165]	training's auc: 0.929991	valid_1's auc: 0.922088
[166]	training's auc: 0.929991	valid_1's auc: 0.922088
[167]	training's auc: 0.930085	valid_1's auc: 0.922267
[168]	training's auc: 0.930085	valid_1's auc: 0.922267
[169]	traini

[304]	training's auc: 0.932963	valid_1's auc: 0.923135
[305]	training's auc: 0.932978	valid_1's auc: 0.92317
[306]	training's auc: 0.932978	valid_1's auc: 0.92317
[307]	training's auc: 0.932987	valid_1's auc: 0.92322
[308]	training's auc: 0.932987	valid_1's auc: 0.92322
[309]	training's auc: 0.933036	valid_1's auc: 0.923266
[310]	training's auc: 0.933036	valid_1's auc: 0.923266
[311]	training's auc: 0.933066	valid_1's auc: 0.923207
[312]	training's auc: 0.933066	valid_1's auc: 0.923207
[313]	training's auc: 0.933082	valid_1's auc: 0.923216
[314]	training's auc: 0.933082	valid_1's auc: 0.923216
[315]	training's auc: 0.933108	valid_1's auc: 0.923153
[316]	training's auc: 0.933108	valid_1's auc: 0.923153
[317]	training's auc: 0.933137	valid_1's auc: 0.923165
[318]	training's auc: 0.933137	valid_1's auc: 0.923165
[319]	training's auc: 0.933185	valid_1's auc: 0.923163
[320]	training's auc: 0.933185	valid_1's auc: 0.923163
[321]	training's auc: 0.933229	valid_1's auc: 0.923161
[322]	training

[455]	training's auc: 0.93501	valid_1's auc: 0.923423
[456]	training's auc: 0.93501	valid_1's auc: 0.923423
[457]	training's auc: 0.935034	valid_1's auc: 0.923443
[458]	training's auc: 0.935034	valid_1's auc: 0.923443
[459]	training's auc: 0.935052	valid_1's auc: 0.92345
[460]	training's auc: 0.935052	valid_1's auc: 0.92345
[461]	training's auc: 0.93506	valid_1's auc: 0.923366
[462]	training's auc: 0.93506	valid_1's auc: 0.923366
[463]	training's auc: 0.935088	valid_1's auc: 0.923337
[464]	training's auc: 0.935088	valid_1's auc: 0.923337
[465]	training's auc: 0.935102	valid_1's auc: 0.923329
[466]	training's auc: 0.935102	valid_1's auc: 0.923329
[467]	training's auc: 0.935125	valid_1's auc: 0.923336
[468]	training's auc: 0.935125	valid_1's auc: 0.923336
[469]	training's auc: 0.935148	valid_1's auc: 0.9233
[470]	training's auc: 0.935148	valid_1's auc: 0.9233
[471]	training's auc: 0.93516	valid_1's auc: 0.923248
[472]	training's auc: 0.93516	valid_1's auc: 0.923248
[473]	training's auc: 

In [51]:
target_test = df_test['30_ROI']
features_test = df_test.drop('30_ROI', axis=1)
y_predict = lgb_b.predict(features_test)

In [52]:
sessoin = pd.DataFrame(y_predict, columns=['pay_30'])
sessoin['pay_30']=sessoin['pay_30'].apply(lambda x:0 if x<0.3 else 1)
y_predict_val = sessoin['pay_30'].values
print(recall_score(target_test, y_predict_val,average=None))
print(precision_score(target_test, y_predict_val))
print(accuracy_score(target_test, y_predict_val))
print(f1_score(target_test, y_predict_val))

[0.76481928 0.87021475]
0.8514007308160779
0.828857791225416
0.860704940741881


In [53]:
target_names = ['0', '1']
print(classification_report(target_test, y_predict_val, target_names=target_names))

              precision    recall  f1-score   support

           0       0.79      0.76      0.78      2075
           1       0.85      0.87      0.86      3213

    accuracy                           0.83      5288
   macro avg       0.82      0.82      0.82      5288
weighted avg       0.83      0.83      0.83      5288



In [54]:
joblib.dump(lgb_b,'./source_lgb_b.pkl')

['./source_lgb_b.pkl']